## Семинар 13: "Обучение с подкреплением 1"

ФИО: Валинуров Денис Юрьевич

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [35]:
import gym
import numpy as np 
from random import randint
#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [4]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [5]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

('initial observation code:', 0)
printing observation:

SFFF
FHFH
FFFH
HFFG
('observations:', Discrete(16), 'n=', 16)
('actions:', Discrete(4), 'n=', 4)


In [6]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
('new observation code:', 4)
('reward:', 0.0)
('is game over?:', False)
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [7]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Попробуйте походить по замерзшему озеру, не упав в дырку. 
* Каждый шаг вы с вероятностью __0.5__ будете двигаться в выбранном направлении и с вероятностью __0.5__ в случайном.
* Если упадете, используйте __env.reset()__ чтобы перезапустить __env__

In [19]:
env.reset()

0

In [20]:
print env.step(action_to_i['right'])
env.render()

(4, 0.0, False, {'prob': 0.3333333333333333})
  (Right)
SFFF
FHFH
FFFH
HFFG


In [21]:
print env.step(action_to_i['right'])
env.render()

(5, 0.0, True, {'prob': 0.3333333333333333})
  (Right)
SFFF
FHFH
FFFH
HFFG


## Задание: 
Следуя шаблонам функций реализуйте алгоритм Policy iteration.
Протестируйте его с помощью функции __evaluate_policy__.
Попробуйте разные значение для gamma и сравните результаты.

In [22]:
def run_episode(env, policy, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [23]:
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [33]:
print env.env.P[0]

{0: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)], 1: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False)], 2: [(0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False)], 3: [(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]}


In [61]:
# Используя фиксированную value_function и жадную стратегию получите policy

def extract_policy(v, gamma = 1.0):
    policy = np.zeros(env.observation_space.n)
    for s in range(env.observation_space.n):
        Q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            Q_sa[a] = sum([prob * (rew + gamma * v[new_s]) for prob, new_s, rew, _ in env.env.P[s][a]])
        policy[s] = np.argmax(Q_sa)
    return policy

In [62]:
# Используя фиксированную policy с помощью сэмплирования получите оценку для value_function с точностью 1e-10


def compute_policy_v(env, policy, gamma=1.0, eps=1e-10):
    
    v = np.zeros(env.observation_space.n)
    prev_v = np.ones(env.observation_space.n)
    while np.sqrt(np.sum((prev_v - v)**2)) > eps:
        prev_v = np.copy(v)
        for s in range(env.observation_space.n):
            v[s] = sum([prob * (rew + gamma * prev_v[new_s]) for prob, new_s, rew, _ in env.env.P[s][policy[s]]])

    return v

In [63]:
def policy_iteration(env, gamma = 1.0):
    policy = np.array([randint(0, 3) for _ in range(env.observation_space.n)])
    max_iterations = 100000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            break

        policy = new_policy
    return policy

In [67]:
for gamma in (0.9, 0.95, 0.99, 1.0):
    env = gym.make("FrozenLake-v0")
    optimal_policy = policy_iteration(env, gamma=gamma)
    policy_score = evaluate_policy(env, optimal_policy, gamma, n=1000)

    print(policy_score)

0.06452433734366068
0.18467415220523156
0.5374010238538228
0.723
